In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [4]:
data = pd.read_csv("Churn_Modelling.csv")

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Preporcess the data

data = data.drop(['RowNumber', 'CustomerId','Surname'], axis=1 )

In [7]:
# Encode Categorical variable
label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [11]:
# Onehot encoder for Geography
onehot_encoder_geo = OneHotEncoder()

geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
# Combine one-hot encoded columns with original data

data = pd.concat([data.drop('Geography', axis = 1 ), geo_encoded_df], axis = 1 )

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
#Split the data into independent and dependent features 

X = data.drop('Exited', axis=1)
y = data['Exited']

In [30]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#Scale theese features
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [31]:
# Save the encoders and scaler for later use

with open('rlabel_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ronehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('rscaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [32]:
# ANN with Regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [33]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1) # Output layer for regression
])

# Compile the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mae'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Set up tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [35]:
# Set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)   

In [36]:
# Trian the Model
history = model.fit(
    X_train,y_train, 
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)


Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2667 - mae: 0.2667 - val_loss: 0.2302 - val_mae: 0.2302
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2157 - mae: 0.2157 - val_loss: 0.2055 - val_mae: 0.2055
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1986 - mae: 0.1986 - val_loss: 0.1906 - val_mae: 0.1906
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1882 - mae: 0.1882 - val_loss: 0.1821 - val_mae: 0.1821
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1818 - mae: 0.1818 - val_loss: 0.1781 - val_mae: 0.1781
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.1774 - mae: 0.1774 - val_loss: 0.1749 - val_mae: 0.1749
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1728 - mae: 0.1728 - val_loss: 0.1704 - val_mae: 0.1704
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - 

In [37]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 15844), started 0:00:11 ago. (Use '!kill 15844' to kill it.)

In [42]:
## Model Evaluation
test_loss, test_mae = model.evaluate(X_test, y_test)

print(f"Test MAE: {test_mae}")


63/63 [==============================] - 0s 3ms/step - loss: 0.1402 - mae: 0.1402
Test MAE: 0.14018753170967102
